# 使用TensorFlow Lite Model Maker生成图像分类器模型

## 预备工作
首先安装程序运行必备的一些库。

In [1]:
!pip install -i https://pypi.tuna.tsinghua.edu.cn/simple/ tflite-model-maker --user

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple/


这里pip之前添加"!"符号是告诉notebook把pip安装指令当做shell指令执行（实际上其实不加！也能执行）。安装的时候遇到`ERROR: Cannot uninstall 'llvmlite'.`的问题。首先卸载llvmlite包，这里利用Anaconda Navigator中Environments组件管理和卸载相关的Package。
![1](https://raw.githubusercontent.com/November-0/Software-project-R-amp-D-practice/main/experiment5/images/1.png)
解决之后，再次提示`conda-repo-cli 1.0.4`和`anaconda-project 0.10.1`没有安装。
分别使用`pip install conda-repo-cli==1.0.4`和`pip install anaconda-project==0.10.1`安装相应的库。注意，由于Anaconda版本不同，安装tflite model maker的环境不同，需根据实际情况自行解决安装过程中遇到的问题。

In [2]:
!pip install -i https://pypi.tuna.tsinghua.edu.cn/simple/ conda-repo-cli==1.0.4

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple/


In [3]:
!pip install conda-repo-cli==1.0.4 -i http://pypi.douban.com/simple/ --trusted-host pypi.douban.com

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: http://pypi.douban.com/simple/


接下来，导入相关的库。

In [4]:
import os

import numpy as np

import tensorflow as tf
assert tf.__version__.startswith('2')

from tflite_model_maker import model_spec
from tflite_model_maker import image_classifier
from tflite_model_maker.config import ExportFormat
from tflite_model_maker.config import QuantizationConfig
from tflite_model_maker.image_classifier import DataLoader

import matplotlib.pyplot as plt


## 模型训练
### 获取数据
先从较小的数据集开始训练，当然越多的数据，模型精度更高。

In [5]:
image_path = tf.keras.utils.get_file(
      'flower_photos.tgz',
      'https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz',
      extract=True)
image_path = os.path.join(os.path.dirname(image_path), 'flower_photos')
print(image_path)

C:\Users\16588\.keras\datasets\flower_photos


这里从`storage.googleapis.com`中下载了本实验所需要的数据集。`image_path`可以定制，默认是在用户目录的`.keras\datasets`中。

### 运行示例
一共需4步完成。
第一步：加载数据集，并将数据集分为训练数据和测试数据。

In [6]:
data = DataLoader.from_folder(image_path)
train_data, test_data = data.split(0.9)

INFO:tensorflow:Load image with size: 3670, num_label: 5, labels: daisy, dandelion, roses, sunflowers, tulips.


第二步：训练Tensorflow模型

In [7]:
# 模型下载到本地，推荐
inception_v3_spec = image_classifier.ModelSpec(uri='D:\juyter\efficientnet_lite0_feature-vector_2')

# 使用在线模型，推荐
# inception_v3_spec = image_classifier.ModelSpec(uri='https://storage.googleapis.com/tfhub-modules/tensorflow/efficientnet/lite0/feature-vector/2.tar.gz')

inception_v3_spec.input_image_shape = [240, 240]
model = image_classifier.create(train_data, model_spec=inception_v3_spec)

INFO:tensorflow:Retraining the models...
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 hub_keras_layer_v1v2 (HubKe  (None, 1280)             3413024   
 rasLayerV1V2)                                                   
                                                                 
 dropout (Dropout)           (None, 1280)              0         
                                                                 
 dense (Dense)               (None, 5)                 6405      
                                                                 
Total params: 3,419,429
Trainable params: 6,405
Non-trainable params: 3,413,024
_________________________________________________________________
None
Epoch 1/5


C:\Users\16588\AppData\Roaming\Python\Python39\site-packages\keras\optimizers\optimizer_v2\gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


103/103 [==============================] - 123s 1s/step - loss: 0.8779 - accuracy: 0.7576
Epoch 2/5
103/103 [==============================] - 127s 1s/step - loss: 0.6552 - accuracy: 0.8999
Epoch 3/5
103/103 [==============================] - 129s 1s/step - loss: 0.6262 - accuracy: 0.9138
Epoch 4/5
103/103 [==============================] - 160s 2s/step - loss: 0.6032 - accuracy: 0.9226
Epoch 5/5
103/103 [==============================] - 157s 2s/step - loss: 0.5923 - accuracy: 0.9329


第三步：评估模型

In [8]:
loss, accuracy = model.evaluate(test_data)

12/12 [==============================] - 22s 1s/step - loss: 0.6336 - accuracy: 0.9019


第四步，导出Tensorflow Lite模型

In [9]:
model.export(export_dir='.')

INFO:tensorflow:Assets written to: C:\Users\16588\AppData\Local\Temp\tmptk9ryh2i\assets


C:\Users\16588\AppData\Roaming\Python\Python39\site-packages\tensorflow\lite\python\convert.py:766: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "


INFO:tensorflow:Label file is inside the TFLite model with metadata.


INFO:tensorflow:Label file is inside the TFLite model with metadata.


INFO:tensorflow:Saving labels in C:\Users\16588\AppData\Local\Temp\tmpynjd4p6c\labels.txt


INFO:tensorflow:Saving labels in C:\Users\16588\AppData\Local\Temp\tmpynjd4p6c\labels.txt


INFO:tensorflow:TensorFlow Lite model exported successfully: .\model.tflite


INFO:tensorflow:TensorFlow Lite model exported successfully: .\model.tflite


这里导出的Tensorflow Lite模型包含了元数据(metadata),其能够提供标准的模型描述。导出的模型存放在Jupyter Notebook当前的工作目录中。
![2](https://raw.githubusercontent.com/November-0/Software-project-R-amp-D-practice/main/experiment5/images/2.png)